In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model
from resnets_utils import *
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
from tensorflow.keras.layers import BatchNormalization

from matplotlib.pyplot import imshow


%matplotlib inline
np.random.seed(1)
tf.random.set_seed(2)

<img src="images/vanishing_grad_kiank.png" style="width:450px;height:220px;">
<caption><center> <u> <font color='purple'>


<a name='3'></a>

## Building a Residual Network

In ResNets, a "shortcut" or a "skip connection" allows the model to skip layers:

The image on the left shows the "main path" through the network. The image on the right adds a shortcut to the main path. By stacking these ResNet blocks on top of each other, you can form a very deep network.

The lecture mentioned that having ResNet blocks with the shortcut also makes it very easy for one of the blocks to learn an identity function. This means that you can stack on additional ResNet blocks with little risk of harming training set performance.

On that note, there is also some evidence that the ease of learning an identity function accounts for ResNets' remarkable performance even more than skip connections help with vanishing gradients.

Two main types of blocks are used in a ResNet, depending mainly on whether the input/output dimensions are the same or different. You are going to implement both of them: the "identity block" and the "convolutional block."


<a name='3-1'></a>

### The Identity Block

The identity block is the standard block used in ResNets, and corresponds to the case where the input activation (say $a^{[l]}$) has the same dimension as the output activation (say $a^{[l+2]}$). To flesh out the different steps of what happens in a ResNet's identity block, here is an alternative diagram showing the individual steps:

<img src="images/idblock2_kiank.png" style="width:650px;height:150px;">

The upper path is the "shortcut path." The lower path is the "main path." In this diagram, notice the CONV2D and ReLU steps in each layer. To speed up training, a BatchNorm step has been added. Don't worry about this being complicated to implement--you'll see that BatchNorm is just one line of code in Keras!

In this exercise, you'll actually implement a slightly more powerful version of this identity block, in which the skip connection "skips over" 3 hidden layers rather than 2 layers. It looks like this:

<img src="images/idblock3_kiank.png" style="width:650px;height:150px;">


First component of main path:

-   The first CONV2D has $F_1$ filters of shape (1,1) and a stride of (1,1). Its padding is "valid". Use 0 as the seed for the random uniform initialization: `kernel_initializer = initializer(seed=0)`.
-   The first BatchNorm is normalizing the 'channels' axis.
-   Then apply the ReLU activation function. This has no hyperparameters.

Second component of main path:

-   The second CONV2D has $F_2$ filters of shape $(f,f)$ and a stride of (1,1). Its padding is "same". Use 0 as the seed for the random uniform initialization: `kernel_initializer = initializer(seed=0)`.
-   The second BatchNorm is normalizing the 'channels' axis.
-   Then apply the ReLU activation function. This has no hyperparameters.

Third component of main path:

-   The third CONV2D has $F_3$ filters of shape (1,1) and a stride of (1,1). Its padding is "valid". Use 0 as the seed for the random uniform initialization: `kernel_initializer = initializer(seed=0)`.
-   The third BatchNorm is normalizing the 'channels' axis.
-   Note that there is **no** ReLU activation function in this component.

Final step:

-   The `X_shortcut` and the output from the 3rd layer `X` are added together.
-   **Hint**: The syntax will look something like `Add()([var1,var2])`
-   Then apply the ReLU activation function. This has no hyperparameters.


In [ ]:
def identity_block(X, f, filters, initializer=random_uniform, training=False):
    """
    Implementation of the identity block as defined in Figure 4

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    initializer -- to set up the initial weights of a layer. Equals to random uniform initializer

    Returns:
    X -- output of the identity block, tensor of shape (m, n_H, n_W, n_C)
    """

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. You'll need this later to add back to the main path.
    X_shortcut = X

    # First component of main path
    X = Conv2D(
        filters=F1,
        kernel_size=1,
        strides=(1, 1),
        padding="valid",
        kernel_initializer=initializer(seed=0),
    )(X)
    X = BatchNormalization(axis=3)(X)  # Default axis
    X = Activation("relu")(X)

    ## Second component of main path (≈3 lines)
    ## Set the padding = 'same'
    X = Conv2D(
        filters=F2,
        kernel_size=f,
        strides=(1, 1),
        padding="same",
        kernel_initializer=initializer(seed=0),
    )(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation("relu")(X)

    ## Third component of main path (≈2 lines)
    ## Set the padding = 'valid'
    X = Conv2D(
        filters=F3,
        kernel_size=1,
        strides=(1, 1),
        padding="valid",
        kernel_initializer=initializer(seed=0),
    )(X)
    X = BatchNormalization(axis=3)(X)

    ## Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X_shortcut, X])
    X = Activation("relu")(X)

    return X

### The Convolutional Block

The ResNet "convolutional block" is the second block type. You can use this type of block when the input and output dimensions don't match up. The difference with the identity block is that there is a CONV2D layer in the shortcut path:

<img src="images/convblock_kiank.png" style="width:650px;height:150px;">

-   The CONV2D layer in the shortcut path is used to resize the input $x$ to a different dimension, so that the dimensions match up in the final addition needed to add the shortcut value back to the main path. (This plays a similar role as the matrix $W_s$ discussed in lecture.)
-   For example, to reduce the activation dimensions's height and width by a factor of 2, you can use a 1x1 convolution with a stride of 2.
-   The CONV2D layer on the shortcut path does not use any non-linear activation function. Its main role is to just apply a (learned) linear function that reduces the dimension of the input, so that the dimensions match up for the later addition step.
-   As for the previous exercise, the additional `initializer` argument is required for grading purposes, and it has been set by default to [glorot_uniform](https://www.tensorflow.org/versions/r2.9/api_docs/python/tf/keras/initializers/GlorotUniform)

The details of the convolutional block are as follows.

First component of main path:

-   The first CONV2D has $F_1$ filters of shape (1,1) and a stride of (s,s). Its padding is "valid". Use 0 as the `glorot_uniform` seed `kernel_initializer = initializer(seed=0)`.
-   The first BatchNorm is normalizing the 'channels' axis.
-   Then apply the ReLU activation function. This has no hyperparameters.

Second component of main path:

-   The second CONV2D has $F_2$ filters of shape (f,f) and a stride of (1,1). Its padding is "same". Use 0 as the `glorot_uniform` seed `kernel_initializer = initializer(seed=0)`.
-   The second BatchNorm is normalizing the 'channels' axis.
-   Then apply the ReLU activation function. This has no hyperparameters.

Third component of main path:

-   The third CONV2D has $F_3$ filters of shape (1,1) and a stride of (1,1). Its padding is "valid". Use 0 as the `glorot_uniform` seed `kernel_initializer = initializer(seed=0)`.
-   The third BatchNorm is normalizing the 'channels' axis. Note that there is no ReLU activation function in this component.

Shortcut path:

-   The CONV2D has $F_3$ filters of shape (1,1) and a stride of (s,s). Its padding is "valid". Use 0 as the `glorot_uniform` seed `kernel_initializer = initializer(seed=0)`.
-   The BatchNorm is normalizing the 'channels' axis.

Final step:

-   The shortcut and the main path values are added together.
-   Then apply the ReLU activation function. This has no hyperparameters.


In [ ]:
def convolutional_block(X, f, filters, s=2, initializer=glorot_uniform, training=False):
    """
    Implementation of the convolutional block as defined in Figure 4

    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    s -- Integer, specifying the stride to be used
    initializer -- to set up the initial weights of a layer. Equals to Glorot uniform initializer,
                   also called Xavier uniform initializer.

    Returns:
    X -- output of the convolutional block, tensor of shape (m, n_H, n_W, n_C)
    """

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    # First component of main path glorot_uniform(seed=0)
    X = Conv2D(
        filters=F1,
        kernel_size=1,
        strides=(s, s),
        padding="valid",
        kernel_initializer=initializer(seed=0),
    )(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation("relu")(X)

    ## Second component of main path (≈3 lines)
    #     X = Conv2D(filters = F2, kernel_size = f, strides = (1, 1) padding = "same", kernel_initializer = initializer(seed=0))(X)
    X = Conv2D(
        filters=F2,
        kernel_size=f,
        strides=(1, 1),
        padding="same",
        kernel_initializer=initializer(seed=0),
    )(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation("relu")(X)

    ## Third component of main path (≈2 lines)
    X = Conv2D(
        filters=F3,
        kernel_size=1,
        strides=(1, 1),
        padding="valid",
        kernel_initializer=initializer(seed=0),
    )(X)
    X = BatchNormalization(axis=3)(X)

    ##### SHORTCUT PATH ##### (≈2 lines)
    X_shortcut = Conv2D(
        filters=F3,
        kernel_size=1,
        strides=(s, s),
        padding="valid",
        kernel_initializer=initializer(seed=0),
    )(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut)

    # Final step: Add shortcut value to main path (Use this order [X, X_shortcut]), and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation("relu")(X)

    return X

<a name='4'></a>

## Building First ResNet Model (50 layers)

<img src="images/resnet_kiank.png" style="width:850px;height:150px;">

The details of this ResNet-50 model are:

-   Zero-padding pads the input with a pad of (3,3)
-   Stage 1:
    -   The 2D Convolution has 64 filters of shape (7,7) and uses a stride of (2,2).
    -   BatchNorm is applied to the 'channels' axis of the input.
    -   MaxPooling uses a (3,3) window and a (2,2) stride.
-   Stage 2:
    -   The convolutional block uses three sets of filters of size [64,64,256], "f" is 3, and "s" is 1.
    -   The 2 identity blocks use three sets of filters of size [64,64,256], and "f" is 3.
-   Stage 3:
    -   The convolutional block uses three sets of filters of size [128,128,512], "f" is 3 and "s" is 2.
    -   The 3 identity blocks use three sets of filters of size [128,128,512] and "f" is 3.
-   Stage 4:
    -   The convolutional block uses three sets of filters of size [256, 256, 1024], "f" is 3 and "s" is 2.
    -   The 5 identity blocks use three sets of filters of size [256, 256, 1024] and "f" is 3.
-   Stage 5:
    -   The convolutional block uses three sets of filters of size [512, 512, 2048], "f" is 3 and "s" is 2.
    -   The 2 identity blocks use three sets of filters of size [512, 512, 2048] and "f" is 3.
-   The 2D Average Pooling uses a window (pool_size) of shape (2,2).
-   The 'flatten' layer doesn't have any hyperparameters.
-   The Fully Connected (Dense) layer reduces its input to the number of classes using a softmax activation.


In [ ]:
def ResNet50(input_shape=(64, 64, 3), classes=6, training=False):
    """
    Stage-wise implementation of the architecture of the popular ResNet50:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> FLATTEN -> DENSE

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation("relu")(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], s=1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    ## Stage 3
    # `convolutional_block` with correct values of `f`, `filters` and `s` for this stage
    X = convolutional_block(X, f=3, filters=[128, 128, 512], s=2)
    #     X =

    # the 3 `identity_block` with correct values of `f` and `filters` for this stage
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])

    # Stage 4
    # add `convolutional_block` with correct values of `f`, `filters` and `s` for this stage
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], s=2)

    # the 5 `identity_block` with correct values of `f` and `filters` for this stage
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    # Stage 5
    # add `convolutional_block` with correct values of `f`, `filters` and `s` for this stage
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], s=2)

    # the 2 `identity_block` with correct values of `f` and `filters` for this stage
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    # AVGPOOL ."X = AveragePooling2D()(X)"
    X = AveragePooling2D(pool_size=(2, 2))(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation="softmax", kernel_initializer=glorot_uniform(seed=0))(
        X
    )

    # Create model
    model = Model(inputs=X_input, outputs=X)

    return model

Run the following code to build the model's graph. If your implementation is incorrect, you'll know it by checking your accuracy when running `model.fit(...)` below.


In [ ]:
# tf.keras.backend.set_learning_phase(True)

model = ResNet50(input_shape=(64, 64, 3), classes=6)
print(model.summary())

In [ ]:
np.random.seed(1)
tf.random.set_seed(2)
opt = tf.keras.optimizers.Adam(learning_rate=0.00015)
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])

The model is now ready to be trained. The only thing you need now is a dataset!


Let's load your old friend, the SIGNS dataset.

<img src="images/signs_data_kiank.png" style="width:450px;height:250px;">
<caption><center> <u> <font color='purple'>


In [ ]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig / 255.0
X_test = X_test_orig / 255.0

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print("number of training examples = " + str(X_train.shape[0]))
print("number of test examples = " + str(X_test.shape[0]))
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(Y_train.shape))
print("X_test shape: " + str(X_test.shape))
print("Y_test shape: " + str(Y_test.shape))

In [ ]:
model.fit(X_train, Y_train, epochs=20, batch_size=32)

In [ ]:
preds = model.evaluate(X_test, Y_test)
print("Loss = " + str(preds[0]))
print("Test Accuracy = " + str(preds[1]))